In [1]:
from models.DistMult import DistMult
from models.Complex import Complex
from models.ConvE import ConvE, ConvE_args

from utils.loaders import load_data, get_onehots
from utils.evaluation_metrics import SRR, auprc_auroc_ap

import torch
import numpy as np
from sklearn.utils import shuffle

In [3]:
from utils.path_manage import get_files

data, lookup, ASD_dictionary, BCE_dictionary = get_files()
entities = int(len(lookup)/2)

In [3]:
number_of_batches = 5
number_of_epochs = 20
x = shuffle(data)
test_data = x[:50] #just limit data to the first n rows

In [8]:
# split your data 
# train_test_eval_split = 

In [5]:
all_objects, all_relationships, all_subjects = np.split(test_data, 3, axis=1)
all_objects = torch.LongTensor(all_objects)
all_subjects = torch.squeeze(torch.LongTensor(all_subjects))
all_relationships = torch.LongTensor(all_relationships)

In [ ]:
model = DistMult(num_entities = entities, embedding_dim=100, num_relations=4)
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)

In [6]:

for epoch in range(number_of_epochs):


        #training stage 
        model.train()
        objects, subjects, relationships  = load_data(test_data, number_of_batches)

        for index in range(number_of_batches):

            obj = torch.LongTensor(objects[index])
            rel = torch.LongTensor(relationships[index])
            subj = torch.squeeze(torch.LongTensor(subjects[index]))

            optimiser.zero_grad()
            pred = model.forward(e1 = obj, rel = rel)
            loss = model.loss(pred, subj)
            loss.backward()
            optimiser.step()


        #evaluation stage
        model.eval()
        objects, subjects, relationships  = load_data(test_data, number_of_batches)
        total_sum_reciporical_rank = torch.zeros(1)

        for index in range(number_of_batches):

            obj = torch.LongTensor(objects[index])
            rel = torch.LongTensor(relationships[index])
            targets = torch.LongTensor(subjects[index])

            predictions = model.forward(e1 = obj, rel = rel)
            srr = SRR(predictions, targets) 
            total_sum_reciporical_rank = total_sum_reciporical_rank + srr

        print('mean reciporical rank is...')
        print(total_sum_reciporical_rank / len(test_data))



mean reciporical rank is...
tensor([0.0002])
mean reciporical rank is...
tensor([0.0004])
mean reciporical rank is...
tensor([0.0009])
mean reciporical rank is...
tensor([0.0041])
mean reciporical rank is...
tensor([0.0229])
mean reciporical rank is...
tensor([0.0340])
mean reciporical rank is...
tensor([0.0869])
mean reciporical rank is...
tensor([0.1516])
mean reciporical rank is...
tensor([0.2423])
mean reciporical rank is...
tensor([0.3343])
mean reciporical rank is...
tensor([0.4514])
mean reciporical rank is...
tensor([0.5638])
mean reciporical rank is...
tensor([0.6878])
mean reciporical rank is...
tensor([0.8322])
mean reciporical rank is...
tensor([0.8922])
mean reciporical rank is...
tensor([0.9297])
mean reciporical rank is...
tensor([0.9564])
mean reciporical rank is...
tensor([1.])
mean reciporical rank is...
tensor([1.])
mean reciporical rank is...
tensor([1.])


In [ ]:
one_hots = get_onehots(targets, entities)
auprc, auroc, ap = auprc_auroc_ap(one_hots, predictions)

In [9]:
print(auroc)
print(auprc)
print(ap)

0.9999998582645916
0.9904545454545454
0.9909090909090909
